## Fake News Classifier Using Bidirectional LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/ISM PROJ/Datasets/train.csv')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df=df.dropna()

In [ ]:
X=df.drop('label',axis=1)

In [ ]:
y=df['label']

In [ ]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [ ]:
X.shape

(18285, 4)

In [ ]:
y.shape

(18285,)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.8.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [ ]:
voc_size=5000

### Onehot Representation

In [ ]:
messages=X.copy()

In [ ]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('all')

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4141, 1067, 145, 802, 1962, 3178, 3959, 111, 3641, 258],
 [2188, 1168, 187, 3053, 4481, 2551, 361],
 [2425, 1122, 95, 4131],
 [3026, 4632, 1009, 3002, 3165, 3789],
 [19, 4481, 2136, 2689, 3610, 3401, 4481, 2536, 3198, 2762],
 [2384,
  2930,
  3609,
  3769,
  3548,
  2963,
  4004,
  4246,
  4767,
  1786,
  2892,
  3234,
  4940,
  4231,
  361],
 [4014, 4037, 1708, 1812, 4000, 223, 3053, 310, 192, 3543, 4124],
 [1437, 3956, 1425, 316, 3297, 2210, 2963, 2741, 192, 3543, 4124],
 [1904, 4060, 3840, 180, 438, 3010, 4467, 4915, 2963, 4312],
 [542, 828, 1623, 4487, 4389, 627, 4748, 1507],
 [4884, 887, 1216, 911, 2167, 2340, 877, 1600, 4928, 1446, 858],
 [3002, 4313, 1962, 3010, 2963, 3297],
 [4905, 28, 2405, 1664, 3639, 1126, 3519, 301, 3481],
 [3615, 1567, 3577, 1634, 2447, 3627, 2581, 192, 3543, 4124],
 [4437, 1552, 3217, 746, 3191, 192, 3543, 4124],
 [2189, 4171, 1759, 1972, 642, 4826, 586, 3587, 107, 218],
 [2005, 3201, 1168],
 [4012, 1578, 2829, 347, 2963, 3965, 55, 361],
 [3757, 2127, 1

### Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  111 3641  258]
 [   0    0    0 ... 4481 2551  361]
 [   0    0    0 ... 1122   95 4131]
 ...
 [   0    0    0 ...  192 3543 4124]
 [   0    0    0 ... 2901  922 4078]
 [   0    0    0 ... 1535 4318  772]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4141,
       1067,  145,  802, 1962, 3178, 3959,  111, 3641,  258], dtype=int32)

In [ ]:
## Creating model
embedding_vector_features=40

model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(18285, (18285,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=101)

### Model Training

In [ ]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 20s 85ms/step - loss: 0.3556 - accuracy: 0.8365 - val_loss: 0.2113 - val_accuracy: 0.9085
Epoch 2/10
192/192 [==============================] - 15s 79ms/step - loss: 0.1650 - accuracy: 0.9328 - val_loss: 0.2208 - val_accuracy: 0.9165
Epoch 3/10
192/192 [==============================] - 15s 79ms/step - loss: 0.1178 - accuracy: 0.9561 - val_loss: 0.2196 - val_accuracy: 0.9152
Epoch 4/10
192/192 [==============================] - 15s 78ms/step - loss: 0.0872 - accuracy: 0.9679 - val_loss: 0.2604 - val_accuracy: 0.9122
Epoch 5/10
192/192 [==============================] - 15s 79ms/step - loss: 0.0591 - accuracy: 0.9800 - val_loss: 0.3396 - val_accuracy: 0.9150
Epoch 6/10
192/192 [==============================] - 15s 79ms/step - loss: 0.0387 - accuracy: 0.9870 - val_loss: 0.2944 - val_accuracy: 0.9130
Epoch 7/10
192/192 [==============================] - 15s 78ms/step - loss: 0.0298 - accuracy: 0.9906 - val_loss: 0.3658 - val_accuracy:

### Performance Metrics And Accuracy

In [ ]:
# y_pred1=model1.predict_classes(X_test)
y_pred = model.predict(X_test)

y_pred1 = np.argmax(y_pred,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred1)

array([[3419,    0],
       [2616,    0]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.5665285832642917

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.57      1.00      0.72      3419
           1       0.00      0.00      0.00      2616

    accuracy                           0.57      6035
   macro avg       0.28      0.50      0.36      6035
weighted avg       0.32      0.57      0.41      6035



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
